# Setup

On google colab, you have to restart runtime after running the following line

In [1]:
!pip install omegaconf

In [2]:
from google.colab import drive
drive.mount("/content/drive/")
#"/content/drive/My Drive/NN-kNN/"
folder_name = "/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/"
import sys
sys.path.insert(0,folder_name)

Mounted at /content/drive/


In [3]:
##This is added because my Rdata uses Cdata for the covid data set.
##Rdata use Cdata function to load the data set, then convert it to regression problem
import os
import sys
sys.path.append('/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset')


In [4]:
# folder_name = os.getcwd()

In [5]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from omegaconf import DictConfig, OmegaConf

from dataset import cls_small_data as Cdata
import model.cls_model as Cmodel
from dataset import cls_medium_data

from dataset import reg_data as Rdata
import model.reg_model as Rmodel

In [6]:
conf_file = OmegaConf.load(os.path.join(folder_name, 'config.yaml'))

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
torch.cuda.is_available()

True

# NCA and LMNN setup

In [9]:
pip install metric-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.6 MB/s eta 0:00:00


In [10]:
import metric_learn
from metric_learn import LMNN,NCA

# Data Sets

Supported small dataset for classification:  
'zebra',
'zebra_special',
'bal',
'digits',
'iris',
'wine',
'breast_cancer',

for regression:
'califonia_housing',
'abalone',
'diabets',
'body_fat',
'ziweifaces'


Newly added data sets for mental health (psychology):

Classification:
'psych_depression_physical_symptons',
'covid_anxious',
'covid_depressed'


In [79]:
dataset_name = 'covid_depressed'
cfg = conf_file['dataset'][dataset_name]
#TODO need to add other covid data sets here.
if dataset_name in ['covid_anxious','covid_depressed','covid_physical','covid_lonely','covid_hopeless',
                    'psych_depression_physical_symptons',
                    'zebra','zebra_special','bal','digits','iris','wine','breast_cancer']:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = Cdata.Cls_small_data(dataset_name)
elif dataset_name in []:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = cls_medium_data.Cls_medium_data(dataset_name)
else:
    criterion = torch.nn.MSELoss()
    Xs, ys = Rdata.Reg_data(dataset_name)

Columns in the dataset: Index(['SU_ID', 'P_PANEL', 'NATIONAL_WEIGHT', 'REGION_WEIGHT',
       'NATIONAL_WEIGHT_POP', 'REGION_WEIGHT_POP', 'NAT_WGT_COMB_POP',
       'REG_WGT_COMB_POP', 'P_GEO', 'SOC1',
       ...
       'REGION9', 'P_DENSE', 'MODE', 'LANGUAGE', 'MAIL50', 'RACE1_BANNER',
       'RACE2_BANNER', 'INC_BANNER', 'AGE_BANNER', 'HH_BANNER'],
      dtype='object', length=177)
   SOC1  SOC2A  SOC2B  SOC3A  SOC3B  SOC4A  SOC4B  PHYS8  PHYS1A  PHYS1B  \
0   3.0    4.0    4.0    1.0    1.0    2.0    2.0    3.0     1.0     1.0   
1   3.0    3.0    3.0    1.0    3.0    2.0    2.0    2.0     2.0     2.0   
2   3.0    3.0    3.0    1.0    1.0    2.0    1.0    2.0     2.0     1.0   
3   3.0    2.0    2.0    1.0    1.0    2.0    1.0    3.0     2.0     2.0   
4   3.0    1.0    1.0    1.0    1.0    2.0    2.0    3.0     1.0     1.0   
5   2.0    3.0    3.0    1.0    1.0    1.0    1.0    2.0     2.0     2.0   
6   2.0    4.0    3.0    1.0    1.0    2.0    2.0    NaN     1.0     2.0   
7   3

In [58]:
# This section is used to reload the imported module.
# For example, if you made any changes in the model.cls_model, you should run importlib.reload(Cmodel) as long as you set import model.cls_model as Cmodel.
import importlib
importlib.reload(Rdata)
importlib.reload(Cdata)

<module 'dataset.cls_small_data' from '/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset/cls_small_data.py'>

In [80]:
#for reloading config file, in case you modified it for experimenting
conf_file = OmegaConf.load(os.path.join(folder_name, 'config.yaml'))
cfg = conf_file['dataset'][dataset_name]

In [81]:
cfg

{'ca_weight_sharing': True, 'top_case_enabled': False, 'training_epochs': 1000, 'learning_rate': 0.01, 'batch_size': 16, 'top_k': 5, 'class_weight_sharing': True, 'patience': 40, 'discount': 40}

# Classification with NNKNN

In [82]:
# prompt: get the unique y values and their counts

unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")
print(f"Xs.size(): {Xs.size()}")


Unique values: [0 1 2 3]
Counts: [776 776 776 776]
Xs.size(): torch.Size([3104, 161])


In [83]:
def train_cls(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)

  # Train model
  # X_train_trimmed = X_train[:, :X_train.shape[1]//10]
  # y_train_trimmed = y_train[:X_train.shape[1]//10]
  model = Cmodel.NN_k_NN(X_train,
              y_train,
              cfg.ca_weight_sharing,
              cfg.top_case_enabled,
              cfg.top_k,
              cfg.discount,
              device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    epoch_msg = True

    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, output, predicted_class = model(X_train_batch)
      loss = criterion(output, y_train_batch)

      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

        epoch_msg = False
      # print("evaluating")
    model.eval()
    with torch.no_grad():
      _, _, output, predicted_class = model(X_test)

      # Calculate accuracy
      accuracy_temp = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)

    elif accuracy_temp > best_accuracy:
      #memorize best model
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0

    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  return best_accuracy, model

In [84]:
def load_model_cls(X_train,y_train,cfg):
  # Define the model architecture
  model = Cmodel.NN_k_NN(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model

In [88]:
accuracies = []
knn_accuracies = []
lmnn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/classifier_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle=True, random_state = None)
enable_lmnn = False

for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  if(enable_lmnn):
    # https://contrib.scikit-learn.org/metric-learn/supervised.html#lmnn
    lmnn = LMNN(n_neighbors=5, learn_rate=1e-6)
    ##TODO, change here if you need to use a different one
    # lmnn = metric_learn.MLKR()
    # lmnn = metric_learn.NCA(max_iter=1000)
    lmnn.fit(X_train,y_train)
    knn = KNeighborsClassifier(n_neighbors=5,metric=lmnn.get_metric())
    knn.fit(X_train,y_train)
    # klmnn_accuracies.append( accuracy_score(knn.predict(X_test), y_test))
    lmnn_acc = accuracy_score(knn.predict(X_test), y_test)
    lmnn_accuracies.append(lmnn_acc)
    print("lmnn_acc: ",lmnn_acc)
    # continue

  knn =  KNeighborsClassifier(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_acc  = accuracy_score(knn.predict(X_test), y_test)
  knn_accuracies.append(knn_acc)
  print("knn_acc: ", knn_acc)

  best_accuracy, model = train_cls(X_train,y_train, X_test, y_test, cfg)
  accuracies.append(best_accuracy)
  print("nnknn acc: ", best_accuracy)


print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")


knn_acc:  0.3858520900321543
Epoch [2/1000], Loss: 1.7401
Epoch [4/1000], Loss: 1.7268
Epoch [6/1000], Loss: 1.2174
Epoch [8/1000], Loss: 1.0451
Epoch [10/1000], Loss: 1.3573
Epoch [12/1000], Loss: 1.0394
Epoch [14/1000], Loss: 1.0319
Epoch [16/1000], Loss: 0.9358
Epoch [18/1000], Loss: 1.1416
Epoch [20/1000], Loss: 1.1829
Epoch [22/1000], Loss: 1.2687
Epoch [24/1000], Loss: 0.9937
Epoch [26/1000], Loss: 0.9522
Epoch [28/1000], Loss: 1.2141
Epoch [30/1000], Loss: 1.1778
Epoch [32/1000], Loss: 0.9218
Epoch [34/1000], Loss: 1.0311
Epoch [36/1000], Loss: 0.8487
Epoch [38/1000], Loss: 1.1949
Epoch [40/1000], Loss: 1.0722
Epoch [42/1000], Loss: 0.9713
Epoch [44/1000], Loss: 0.9606
Epoch [46/1000], Loss: 0.9046
Epoch [48/1000], Loss: 0.8502
Epoch [50/1000], Loss: 0.8455
Epoch [52/1000], Loss: 0.6412
Epoch [54/1000], Loss: 0.8090
Epoch [56/1000], Loss: 0.9169
Epoch [58/1000], Loss: 0.8580
Epoch [60/1000], Loss: 0.8090
Epoch [62/1000], Loss: 1.0744
Epoch [64/1000], Loss: 0.9520
Epoch [66/1000]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [89]:
print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")

Average accuracy:0.507
KNN accuracy:0.343
LMNN/NCA accuracy:nan


# Regression with NNKNN

In [71]:
unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")


Unique values: [-1.3401558 -0.4467186  0.4467186  1.3401558]
Counts: [113 113 113 113]


In [72]:
def train_reg(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=cfg.batch_size, shuffle=False)


    # Train model
  model = Rmodel.NN_k_NN_regression(X_train,
                                    y_train,
                                    cfg.ca_weight_sharing,
                                    cfg.top_case_enabled,
                                    cfg.top_k,
                                    cfg.discount,
                                    cfg.class_weight_sharing,
                                    device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    # break # no training
    epoch_msg = True
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, _, predicted_number = model(X_train_batch)
      # break
      loss = criterion(predicted_number.squeeze(), y_train_batch)
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

    model.eval()
    with torch.no_grad():
      predicted_numbers = []
      for X_test_batch, _ in test_loader:
        X_test_batch = X_test_batch.to(device)
        _, _, _, predicted_number = model(X_test_batch)
        predicted_numbers.extend(predicted_number.squeeze().cpu().detach())

      predicted_numbers = torch.Tensor(predicted_numbers)
      accuracy_temp = criterion(y_test, predicted_numbers)

    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)
    elif accuracy_temp < best_accuracy:
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0
    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  _, case_activations, topk_avg, predicted_number = model(X_test)

  top_case_indices = torch.topk(case_activations, 5, dim=1)[1].cpu()

  accuracy = criterion(y_test, predicted_number.squeeze().cpu())
  y_train = y_train.cpu()
  top_k_average_accuracy = mean_squared_error(torch.mean(y_train[top_case_indices], dim=1), y_test)
  # top_k_average_accuracy = mean_squared_error(topk_avg.cpu(), y_test)
  return best_accuracy, accuracy, top_k_average_accuracy, model

In [73]:
# prompt: load_model_reg()

def load_model_reg(X_train,y_train,cfg):
  # Define the model architecture
  model = Rmodel.NN_k_NN_regression(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      cfg.class_weight_sharing,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model


In [74]:
best_accuracies = []
accuracies = []
top_k_average_accuracies = []
knn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/regression_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle = True,random_state = None)


for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]

  knn = KNeighborsRegressor(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_accuracies.append(mean_squared_error(knn.predict(X_test), y_test))

  best_accuracy, accuracy, top_k_average_accuracy, model= train_reg(X_train, y_train, X_test, y_test, cfg)
  best_accuracies.append(best_accuracy)
  accuracies.append(accuracy)
  top_k_average_accuracies.append(top_k_average_accuracy)
  # break

print("Average accuracy:", np.mean([acc.detach().numpy() for acc in accuracies]))
print("Average top_k_average_accuracies", np.mean(top_k_average_accuracies))
print("KNN accuracy:", np.mean(knn_accuracies))

Epoch [2/1000], Loss: 0.9101
Epoch [4/1000], Loss: 1.1636
Epoch [6/1000], Loss: 1.1765
Epoch [8/1000], Loss: 0.8428
Epoch [10/1000], Loss: 1.1137
Epoch [12/1000], Loss: 1.0292
Epoch [14/1000], Loss: 1.0982
Epoch [16/1000], Loss: 0.9925
Epoch [18/1000], Loss: 1.0336
Epoch [20/1000], Loss: 0.9227
Epoch [22/1000], Loss: 0.7754
Epoch [24/1000], Loss: 1.3384
Epoch [26/1000], Loss: 1.0632
Epoch [28/1000], Loss: 1.3018
Epoch [30/1000], Loss: 1.1665
Epoch [32/1000], Loss: 1.2365
Epoch [34/1000], Loss: 1.2915
Epoch [36/1000], Loss: 1.0312
Epoch [38/1000], Loss: 1.0357
Epoch [40/1000], Loss: 1.0027
Epoch [42/1000], Loss: 1.1192
Epoch [44/1000], Loss: 1.1279
Epoch [46/1000], Loss: 1.3108
Epoch [48/1000], Loss: 0.7497
Epoch [50/1000], Loss: 0.9951
Epoch [52/1000], Loss: 0.9586
Epoch [54/1000], Loss: 1.0915
Epoch [56/1000], Loss: 1.0227
Epoch [58/1000], Loss: 0.8132
Epoch [60/1000], Loss: 1.0798
Epoch [62/1000], Loss: 1.1221
Epoch [64/1000], Loss: 1.1231
Epoch [66/1000], Loss: 1.1144
Epoch [68/1000

In [75]:
print("Average accuracy:", np.mean([acc.detach().numpy() for acc in accuracies]))
print("Average top_k_average_accuracies", np.mean(top_k_average_accuracies))
print("KNN accuracy:", np.mean(knn_accuracies))

Average accuracy: 0.68093604
Average top_k_average_accuracies 0.80702674
KNN accuracy: 0.96628094


# Results Interpretation

In [52]:
def print_model_features(input_model):
  for n, p in model.named_parameters():
    print(n)
    print(p.data)

In [53]:
print_model_features(model)

fa_layer.f1weight
tensor([2.1106, 1.8908, 1.6924, 1.8710, 1.0601, 2.5825, 2.9094, 2.5395, 2.0066,
        1.9016, 1.7142, 2.1770, 1.9348, 2.3703, 1.6598, 2.0052, 2.3584, 1.6951,
        1.6328, 1.8538, 1.6562, 1.6381, 2.1498, 1.9748, 1.9203, 3.1412, 3.0589,
        2.8568, 2.0025, 2.6408, 2.2499, 1.5575, 2.3280, 1.4291, 2.4704, 2.5024,
        1.8397, 2.4717, 1.6664, 2.6849, 2.9335, 2.1258, 2.9493, 1.7068, 1.9431,
        1.3820, 1.5805, 2.0954, 1.1535, 0.4391, 1.0000, 1.4104, 1.9261, 1.6943,
        1.4370, 1.4291, 2.3542, 2.8033, 2.3593, 2.4285, 2.3041, 2.9973, 2.5918,
        2.3669, 2.4644, 2.3748, 2.8236, 2.7546, 2.6256, 2.7163, 2.3899, 2.9882,
        3.0433, 2.7108, 2.7331, 2.8388, 2.0365, 3.2152, 0.8640, 1.4741, 1.0156,
        1.4397, 1.4917, 1.6752, 1.7452, 1.3472, 0.6947, 0.4216, 0.5387, 0.6400,
        1.8314, 1.7414, 2.2853, 0.9081, 0.2798, 1.7630, 2.0298, 1.7335, 1.7092,
        0.8255, 1.0003, 0.7772, 1.0696, 1.3882, 1.9084, 2.1599, 0.9502, 0.9828,
        0.8587, 1.3444

In [54]:
# for regression only. for classification is different
#feature_activations, case_activations, predicted_number
model.eval()
feature_activations, case_activations, output, predicted_class = model(X_test.to(device))

In [55]:
predicted_class

tensor([ 0.8999, -0.8666, -0.9217, -0.9256,  0.5683, -1.0220,  1.2581,  1.0604,
        -0.5911,  1.2263,  0.7885, -0.6577, -0.6736,  0.6268, -0.4529,  0.7075,
         0.1268, -0.7729, -0.5187,  1.0548,  0.9817,  0.9292, -0.8060, -0.2730,
        -0.9373, -0.8038, -0.6712, -0.8421,  0.1013,  0.8680,  0.3269,  1.1277,
        -0.5450,  1.0284, -0.4112,  0.4573, -0.1607, -0.2726,  0.0489, -0.6691,
        -0.8145, -0.4316, -0.1176, -0.4313, -0.9800,  0.4471,  0.6697,  0.5955,
        -0.9372, -0.3046, -0.0488,  1.0132, -0.6103,  0.5288,  0.0120,  0.4471,
         0.0000,  0.9239, -0.8645,  0.4044, -0.1234,  1.1789, -0.2887, -0.4899,
        -0.5348,  0.6737,  1.1257, -0.6077, -0.5535,  1.0625, -0.9370, -0.6413,
        -0.8016, -0.7329,  1.2232,  0.5508, -0.6154, -0.8148, -0.5959,  0.1337,
        -0.5411, -0.5397, -0.7780,  0.2527, -0.6770, -0.8956, -0.8376, -0.4424,
         0.3260, -0.5228,  0.9518,  0.4242,  1.0518, -0.6074,  0.0950,  0.4988,
        -0.8511,  0.8652, -0.7863,  0.96

In [56]:
y_test

tensor([ 1.3414, -1.3414, -0.4471, -1.3414, -0.4471, -1.3414,  1.3414,  1.3414,
        -1.3414,  1.3414,  1.3414, -1.3414, -1.3414,  1.3414, -1.3414,  1.3414,
         0.4471, -0.4471, -1.3414,  1.3414,  0.4471,  0.4471, -0.4471,  0.4471,
        -1.3414,  1.3414,  0.4471, -1.3414,  0.4471,  1.3414,  0.4471,  1.3414,
        -1.3414,  1.3414, -1.3414,  1.3414, -0.4471,  0.4471,  0.4471,  0.4471,
        -1.3414,  1.3414, -0.4471, -0.4471, -1.3414,  1.3414,  1.3414,  1.3414,
        -1.3414,  1.3414,  0.4471,  1.3414,  1.3414,  0.4471, -1.3414,  1.3414,
        -1.3414,  1.3414, -1.3414, -1.3414,  0.4471,  1.3414, -1.3414, -1.3414,
         0.4471,  1.3414,  1.3414,  1.3414, -1.3414,  1.3414, -1.3414, -0.4471,
        -0.4471, -0.4471,  1.3414,  1.3414, -0.4471,  1.3414,  1.3414, -0.4471,
         1.3414,  0.4471, -1.3414,  0.4471, -0.4471, -0.4471, -1.3414, -0.4471,
         0.4471, -1.3414,  1.3414,  0.4471,  0.4471,  0.4471,  0.4471,  0.4471,
         1.3414,  1.3414, -1.3414,  1.34

In [57]:
# prompt: accuracy comparing predicted_class and y_test

accuracy = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
print("Accuracy:", accuracy)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
#inspecting the case activations
top_case_indices = torch.topk(case_activations, 5, dim=1)[1]

In [ ]:
X_test[0]

In [ ]:
y_test[0]

In [ ]:
X_train[top_case_indices[0][0]]

In [ ]:
y_train[top_case_indices[0][0]]

By comparing the following two blocks' outputs, you can see we are retrieving a good neighbor.

In [ ]:
#sum abs of X_test[0] and the top activated case
sum(abs(X_test[0] - X_train[top_case_indices[0][0]]))

In [ ]:
# prompt: average sum abs of X_test[0] and X_train data
print(np.mean([sum(abs(X_test[0] - X_train[i])) for i in range(len(X_train))]))

TODO:: A better way is to show the distribution of ``X_test[0] - X_train[i]``

In [ ]:
y_train[top_case_indices[0]]

In [ ]:
knn.predict(X_test)[0]

In [ ]:
indices = knn.kneighbors(X_test)[1][0]

In [ ]:
y_train[indices]

# Sanity Check


## Classification Neural Network

In [ ]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 1024
num_classes = torch.unique(ys).shape[0]
learning_rate = 1e-5
batch_size = 16
epochs = 2000

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the neural network architecture for classification
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(input_size, hidden_size ),
            nn.LeakyReLU(),
            nn.Linear(hidden_size , hidden_size // 2),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 4, num_classes)
            )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    m.bias.data.fill_(0)

    def forward(self, x):
        return self.nn(x)


In [ ]:
best_accuracies = []
for train_index, test_index in k_fold.split(Xs):
  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
  patience_counter = 0
  best_model = None
  best_accuracy = None
  # Initialize the model, loss function, and optimizer
  model = NeuralNet(input_size, hidden_size, num_classes)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Training loop
  for epoch in range(epochs):
    epoch_msg = True
    training_total_acc = 0.0
    training_total_loss = 0.0
    num_of_batches = len(train_loader)
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      # Forward pass
      outputs = model(X_train_batch)
      loss = criterion(outputs, y_train_batch)

      # Backward and optimize
      _, predicted = torch.max(outputs, 1)
      training_total_acc += torch.sum(predicted == y_train_batch).item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      training_total_loss += loss.item()
      # if (i + 1) % 5 == 0
    if epoch == 0 or (epoch + 1) % 100 == 0:
      print(f"Epoch: {epoch + 1}, Training Loss: {training_total_loss/num_of_batches:.2f} Acc: {training_total_acc/num_of_batches:.2f}")
    # Testing the model
    model.eval()
    with torch.no_grad():
      outputs = model(X_test)
      loss = criterion(outputs, y_test)
      _, predicted = torch.max(outputs, 1)
      accuracy = torch.sum(predicted == y_test).item() / len(y_test)
      print(f'Accuracy on the test set: {accuracy * 100:.2f}%')
      if best_accuracy is None or accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        patience_counter = 0
      else:
        patience_counter += 1
      if epoch_msg and (epoch + 1) % 100 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{epoch}], Test Loss: {loss.item()}')
    if patience_counter >= cfg.patience:
      print("Best acc achieved: ", best_accuracy)
      break
  best_accuracies.append(best_accuracy)
  # break
print("Average accuracy:", np.mean(best_accuracies))

Epoch: 1, Training Loss: 1.13 Acc: 4.00
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 58.33%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 58.33%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Acc

## Regression Neural Network

In [76]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 100
# num_classes = torch.unique(ys).shape[0]
learning_rate = 1e-5
batch_size = 16
epochs = 2000

In [77]:
# prompt: a standard neural network with 3 fully connected layers for regression

import torch
import torch.nn as nn
import torch.nn.functional as F

class RegressionNet(nn.Module):
    def __init__(self, input_size):
        super(RegressionNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(input_size, hidden_size ),
            nn.LeakyReLU(),
            nn.Linear(hidden_size , hidden_size // 2),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 4, 1)
            )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    m.bias.data.fill_(0)

    def forward(self, x):
        return self.nn(x).squeeze()

In [78]:
best_accuracies = []
for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
  patience_counter = 0
  best_model = None
  best_accuracy = None
  model = RegressionNet(Xs.shape[1])
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  for epoch in range(epochs):
    epoch_msg = True
    training_total_loss = 0.0
    num_of_batches = len(train_loader)
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      # Forward pass
      outputs = model(X_train_batch)
      loss = criterion(outputs, y_train_batch)
      training_total_loss += loss.item()
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if epoch == 0 or (epoch + 1) % 3 == 0:
      print(f'Epoch: {epoch + 1}, Training Loss: {training_total_loss/num_of_batches:.2f}')

    # Testing the model
    model.eval()
    with torch.no_grad():
      outputs = model(X_test)
      loss = criterion(outputs, y_test)
      if best_accuracy is None or loss.item() < best_accuracy:
        best_accuracy = loss.item()
        best_model = model
        patience_counter = 0
      else:
        patience_counter += 1
      if epoch_msg and (epoch + 1) % 100 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{epochs}], Test Loss: {loss.item()}')
        # print(f'Loss on the test set: {loss.item()}')
    if patience_counter >= cfg.patience:
      print("Best loss achieved: ", best_accuracy)
      break
  best_accuracies.append(best_accuracy)
print("Average accuracy:", np.mean(best_accuracies))


Epoch: 1, Training Loss: 1.67
Epoch: 3, Training Loss: 1.59
Epoch: 6, Training Loss: 1.46
Epoch: 9, Training Loss: 1.37
Epoch: 12, Training Loss: 1.29
Epoch: 15, Training Loss: 1.23
Epoch: 18, Training Loss: 1.16
Epoch: 21, Training Loss: 1.13
Epoch: 24, Training Loss: 1.09
Epoch: 27, Training Loss: 1.06
Epoch: 30, Training Loss: 1.02
Epoch: 33, Training Loss: 1.01
Epoch: 36, Training Loss: 0.99
Epoch: 39, Training Loss: 0.98
Epoch: 42, Training Loss: 0.96
Epoch: 45, Training Loss: 0.96
Epoch: 48, Training Loss: 0.95
Epoch: 51, Training Loss: 0.94
Epoch: 54, Training Loss: 0.94
Epoch: 57, Training Loss: 0.93
Epoch: 60, Training Loss: 0.93
Epoch: 63, Training Loss: 0.92
Epoch: 66, Training Loss: 0.92
Epoch: 69, Training Loss: 0.91
Epoch: 72, Training Loss: 0.91
Epoch: 75, Training Loss: 0.90
Epoch: 78, Training Loss: 0.91
Epoch: 81, Training Loss: 0.91
Epoch: 84, Training Loss: 0.90
Epoch: 87, Training Loss: 0.90
Epoch: 90, Training Loss: 0.88
Epoch: 93, Training Loss: 0.89
Epoch: 96, T